<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0220ENSkillsNetwork900-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


<h1>Extracting and Visualizing Stock Data</h1>
<h2>Description</h2>


Extracting essential data from a dataset and displaying it is a necessary part of data science; therefore individuals can make correct decisions based on the data. In this assignment, you will extract some stock data, you will then display this data in a graph.


<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Define a Function that Makes a Graph</li>
        <li>Question 1: Use yfinance to Extract Stock Data</li>
        <li>Question 2: Use Webscraping to Extract Tesla Revenue Data</li>
        <li>Question 3: Use yfinance to Extract Stock Data</li>
        <li>Question 4: Use Webscraping to Extract GME Revenue Data</li>
        <li>Question 5: Plot Tesla Stock Graph</li>
        <li>Question 6: Plot GameStop Stock Graph</li>
    </ul>
<p>
    Estimated Time Needed: <strong>30 min</strong></p>
</div>

<hr>


In [ ]:
!pip install yfinance==0.1.67
!mamba install bs4==4.10.0 -y
!pip install nbformat==4.2.0

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define Graphing Function


In this section, we define the function `make_graph`. You don't have to know how the function works, you should only care about the inputs. It takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.


In [ ]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Question 1: Use yfinance to Extract Stock Data


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is `TSLA`.


In [ ]:
import yfinance as yf
tesla = yf.Ticker('TSLA')

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `tesla_data`. Set the `period` parameter to `max` so we get information for the maximum amount of time.


In [ ]:
try:
    tesla_data = tesla.history(period="max")
    print(tesla_data.head())  # Verifique se há dados
except Exception as e:
    print("Erro ao obter dados:", e)

**Reset the index** using the `reset_index(inplace=True)` function on the tesla_data DataFrame and display the first five rows of the `tesla_data` dataframe using the `head` function. Take a screenshot of the results and code from the beginning of Question 1 to the results below.


In [ ]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

## Question 2: Use Webscraping to Extract Tesla Revenue Data


Use the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm Save the text of the response as a variable named `html_data`.


In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text

Parse the html data using `beautiful_soup`.


In [ ]:
soup = BeautifulSoup(html_data, "html5lib")
#print(soup.prettify())

Using `BeautifulSoup` or the `read_html` function extract the table with `Tesla Quarterly Revenue` and store it into a dataframe named `tesla_revenue`. The dataframe should have columns `Date` and `Revenue`.


<details><summary>Click here if you need help locating the table</summary>

```
    
Below is the code to isolate the table, you will now need to loop through the rows and columns like in the previous lab
    
soup.find_all("tbody")[1]
    
If you want to use the read_html function the table is located at index 1


```

</details>


In [ ]:
# Criando DataFrame
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

# Extraindo dados
for table in soup.find_all("table"):
    if table.find("th").getText().startswith("Tesla Quarterly Revenue"):
        for row in table.find("tbody").find_all("tr"):
            col = row.find_all("td")
            if len(col) != 2: continue
            Date = col[0].text
            Revenue = col[1].text.replace("$", "").replace(",", "")

            # Usando concat ao invés de append
            tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame([{"Date": Date, "Revenue": Revenue}])], ignore_index=True)

print(tesla_revenue.head())  # Testando a saída

Execute the following line to remove the comma and dollar sign from the `Revenue` column. 


In [ ]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

Execute the following lines to remove an null or empty strings in the Revenue column.


In [ ]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

Display the last 5 row of the `tesla_revenue` dataframe using the `tail` function. Take a screenshot of the results.


In [ ]:
tesla_revenue.tail()

## Question 3: Use yfinance to Extract Stock Data


In [ ]:
import yfinance as yf
print("Sucessfull")

In [ ]:
gme = yf.Ticker('GME')
print("done!")

In [ ]:
try:
    # Obtendo dados históricos
    gme_data = gme.history(period="max")

    # Verificando se os dados foram realmente carregados
    if gme_data.empty:
        print("Erro: Nenhum dado retornado pela API.")
    else:
        print(gme_data.head())  # Exibir as primeiras linhas dos dados

except Exception as e:
    print("Erro ao obter dados:", e)


In [ ]:
gme_data.reset_index(inplace=True)
gme_data.head()

## Question 4: Use Webscraping to Extract GME Revenue Data


Use the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html. Save the text of the response as a variable named `html_data`.


In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data = requests.get(url).text

Parse the html data using `beautiful_soup`.


In [ ]:
soup = BeautifulSoup(html_data, "html5lib")
print(soup.prettify())

Using `BeautifulSoup` or the `read_html` function extract the table with `GameStop Quarterly Revenue` and store it into a dataframe named `gme_revenue`. The dataframe should have columns `Date` and `Revenue`. Make sure the comma and dollar sign is removed from the `Revenue` column using a method similar to what you did in Question 2.


<details><summary>Click here if you need help locating the table</summary>

```
    
Below is the code to isolate the table, you will now need to loop through the rows and columns like in the previous lab
    
soup.find_all("tbody")[1]
    
If you want to use the read_html function the table is located at index 1


```

</details>


In [ ]:
import pandas as pd

# Criar DataFrame vazio
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

# Lista para armazenar os dados antes de criar o DataFrame
data_list = []

for table in soup.find_all('table'):
    if table.find('th').getText().startswith("GameStop Quarterly Revenue"):
        for row in table.find("tbody").find_all("tr"):
            col = row.find_all("td")
            if len(col) != 2: continue
            
            Date = col[0].text
            Revenue = col[1].text.replace("$", "").replace(",", "")

            # Armazena os dados na lista temporária
            data_list.append({"Date": Date, "Revenue": Revenue})

# Criar DataFrame usando pd.concat()
gme_revenue = pd.concat([gme_revenue, pd.DataFrame(data_list)], ignore_index=True)

print(gme_revenue.head())


In [ ]:
gme_revenue.tail()

## Question 5: Plot Tesla Stock Graph


Use the `make_graph` function to graph the Tesla Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(tesla_data, tesla_revenue, 'Tesla')`. Note the graph will only show data upto June 2021.


In [1]:
make_graph(stock_data, revenue_data, 'TSLA')

NameError: name 'stock_data' is not defined

## Question 6: Plot GameStop Stock Graph


Use the `make_graph` function to graph the GameStop Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(gme_data, gme_revenue, 'GameStop')`. Note the graph will only show data upto June 2021.


In [8]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display
import time

TICKER = "GME"  # GameStop stock
ALPHAVANTAGE_API_KEY = "DLTPD7KIZN1B4KIY"  # Replace with your Alpha Vantage API key

# ========== DATA SOURCE 1: yfinance (Primary) ==========
def get_yfinance_data(ticker, start_date, end_date):
    try:
        data = yf.download(
            ticker,
            start=start_date,
            end=end_date,
            progress=False
        ).reset_index()

        if not data.empty:
            print(f"Successfully downloaded {ticker} data from yfinance")
            return data
    except Exception as e:
        print(f"yfinance failed: {str(e)}")
    return None

# ========== DATA SOURCE 2: Alpha Vantage (Fallback) ==========
def get_alphavantage_data(ticker, api_key):
    try:
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={api_key}&datatype=csv&outputsize=full"
        data = pd.read_csv(url)

        # Convert AlphaVantage format to match yfinance
        data = data.rename(columns={
            'timestamp': 'Date',
            'close': 'Close',
            'volume': 'Volume',
            'open' : 'Open',
            'high' : 'High',
            'low' : 'Low'
        })
        data['Date'] = pd.to_datetime(data['Date'])

        print(f"Successfully downloaded {ticker} data from Alpha Vantage")
        return data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    except Exception as e:
        print(f"Alpha Vantage failed: {str(e)}")
        time.sleep(15) # add a delay to avoid rate limit.
        return None

# ========== MAIN DATA LOADING ==========
def load_stock_data(ticker, start_date, end_date):
    # Try yfinance first
    data = get_yfinance_data(ticker, start_date, end_date)

    # If failed, try Alpha Vantage
    if data is None and ALPHAVANTAGE_API_KEY != "YOUR_API_KEY":
        print("Falling back to Alpha Vantage...")
        data = get_alphavantage_data(ticker, ALPHAVANTAGE_API_KEY)

    # If still no data, load sample data
    if data is None:
        print("Using sample data as last resort")
        date_range = pd.date_range(start=start_date, end=end_date)
        num_days = len(date_range)
        data = pd.DataFrame({
            'Date': date_range,
            'Close': [100 + i * 2 + (i % 5) * 3 for i in range(num_days)], # add some noise.
            'Volume': [1000000 + i * 50000 + (i % 7) * 20000 for i in range(num_days)] # add some noise.
        })

    return data

# ========== VISUALIZATION FUNCTION ==========
def make_graph(stock_data, stock, period_name=""):
    # Create figure
    fig = go.Figure()

    # Create subplots
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        subplot_titles=(
            f"{stock} Share Price ({period_name})",
            f"{stock} Trading Volume ({period_name})"
        )
    )

    # Add Price trace
    fig.add_trace(
        go.Scatter(
            x=stock_data['Date'],
            y=stock_data['Close'],
            name="Price",
            line=dict(color='#1f77b4')
        ),
        row=1, col=1
    )

    # Add Volume trace
    fig.add_trace(
        go.Scatter(
            x=stock_data['Date'],
            y=stock_data['Volume'],
            name="Volume",
            line=dict(color='#ff7f0e')
        ),
        row=2, col=1
    )

    # Update layout
    fig.update_layout(
        height=800,
        title_text=f"{stock} Stock Analysis - {period_name}",
        hovermode="x unified"
    )

    # Update y-axes
    fig.update_yaxes(title_text="Price ($USD)", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)

    # Show plot
    display(fig)

# Example Usage to graph GameStop data
start_date = "2024-01-01"
end_date = "2024-12-31"

stock_data = load_stock_data(TICKER, start_date, end_date)

if stock_data is not None:
    make_graph(stock_data, TICKER, "2024")
else:
    print("Failed to retrieve stock data.")

Successfully downloaded GME data from yfinance


<h2>About the Authors:</h2> 

<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0220ENSkillsNetwork900-2022-01-01">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

Azim Hirjani


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description        |
| ----------------- | ------- | ------------- | ------------------------- |
| 2022-02-28        | 1.2     | Lakshmi Holla | Changed the URL of GameStop |
| 2020-11-10        | 1.1     | Malika Singla | Deleted the Optional part |
| 2020-08-27        | 1.0     | Malika Singla | Added lab to GitLab       |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>

<p>
